In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.optimize import least_squares
import math
import pandas as pd
import csv
import datetime
from scipy import stats
from scipy import optimize
from datetime import date, timedelta
import random
import requests
from scipy.stats import truncnorm
import datetime

import statsmodels as sm
#import statsmodels.api as sm

from PIL import Image
from scipy.stats import norm

from statsmodels.tsa.vector_ar.var_model import VAR

import functools

/Users/julianeoliveira/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Read data

## Cases

In [2]:
df = pd.read_csv('/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/data/covid.csv')

## Population

In [3]:
#pop = pd.read_csv('/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/results/seiir_bootstrap/populacao_municipio.csv')

## Mobility data

In [4]:
mob = pd.read_csv('/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/data/distancing_series.csv')

## Stringency

In [5]:
strg = pd.read_csv('/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/data/stringency_series.csv')

# Functions

In [6]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

In [7]:
def crosscorr(datax, datay, lag=0):
    """ Lag-N cross correlation. 
    Parameters
    ----------
    lag : int, default 0
    datax, datay : pandas.Series objects of equal length

    Returns
    ----------
    crosscorr : float
    """
    return datax.corr(datay.shift(lag))

# Create dataframe to analyse the following time series:

    * Daily number of cases
    * Acumulated number of cases
    * SMRI
    * Stringency
    * Rt

In [8]:
df = df[(df.date >= '2020-03-01') & (df.date <= '2020-05-22')]

In [9]:
mob['date2'] = pd.to_datetime(mob.date)

In [10]:
mob.head(2)

,state,name,indUF,indCAP,indInland,date,date2
0,12,AC,33.924855,34.104342,33.770914,01-Mar-2020 00:00:00,2020-03-01
1,12,AC,34.429550,34.609595,34.275130,02-Mar-2020 00:00:00,2020-03-02


In [11]:
mob = mob[(mob.date2 <= '2020-05-22')]

In [12]:
series_state = df.groupby(['state', 'date'])['new_confirmed'].sum().reset_index()

In [13]:
strg['Date2'] = pd.to_datetime(strg.Date)

AttributeError: 'DataFrame' object has no attribute 'Date'

In [14]:
strg = strg[(strg.Date2 <= '2020-05-22')]

AttributeError: 'DataFrame' object has no attribute 'Date2'

In [113]:
strg

,UF,Cod,Date,indGeneral,O1,O2,O3,O4,C1,C2,C3,Date2
0,AC,12,11-Mar-2020 00:00:00,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,2020-03-11
1,AC,12,12-Mar-2020 00:00:00,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,2020-03-12
2,AC,12,13-Mar-2020 00:00:00,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,2020-03-13
3,AC,12,14-Mar-2020 00:00:00,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,2020-03-14
4,AC,12,15-Mar-2020 00:00:00,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,2020-03-15
...,...,...,...,...,...,...,...,...,...,...,...,...
2200,TO,17,18-May-2020 00:00:00,48.809524,0.0,1.0,0.25,0.5,0.5,0.166667,1.0,2020-05-18
2201,TO,17,19-May-2020 00:00:00,48.809524,0.0,1.0,0.25,0.5,0.5,0.166667,1.0,2020-05-19
2202,TO,17,20-May-2020 00:00:00,48.809524,0.0,1.0,0.25,0.5,0.5,0.166667,1.0,2020-05-20
2203,TO,17,21-May-2020 00:00:00,48.809524,0.0,1.0,0.25,0.5,0.5,0.166667,1.0,2020-05-21


In [102]:
s = 'AC'
cases = series_state[series_state.state == s]
mobs = mob[mob.name == s][(mob[mob.name == s].date2 >= cases.iloc[0,1])]
cases

,state,date,new_confirmed
0,AC,2020-03-17,3
1,AC,2020-03-18,0
2,AC,2020-03-19,1
3,AC,2020-03-20,3
4,AC,2020-03-21,4
...,...,...,...
62,AC,2020-05-18,238
63,AC,2020-05-19,248
64,AC,2020-05-20,0
65,AC,2020-05-21,621


In [15]:
lst= []
for s in mob.name.unique():
    
    print(s)
    
    #filtering the state in the datasets
    cases = series_state[series_state.state == s]
    
    mobs = mob[mob.name == s][(mob[mob.name == s].date2 >= cases.iloc[0,1])]
    
    strg = strg[strg.UF == s]
    
    # creating series of cummulative, log and etc
    
    cases['indUF'] = np.array(mobs.indUF)    
    
    lst.append(cases)

AC


AttributeError: 'DataFrame' object has no attribute 'UF'

In [125]:
lst

[   state        date  new_confirmed      indUF
 0     AC  2020-03-17              3  40.871728
 1     AC  2020-03-18              0  42.792925
 2     AC  2020-03-19              1  45.103640
 3     AC  2020-03-20              3  47.939821
 4     AC  2020-03-21              4  50.940040
 ..   ...         ...            ...        ...
 62    AC  2020-05-18            238  50.061087
 63    AC  2020-05-19            248  50.166172
 64    AC  2020-05-20              0  50.332421
 65    AC  2020-05-21            621  50.702349
 66    AC  2020-05-22            240  50.486701
 
 [67 rows x 4 columns],
     state        date  new_confirmed      indUF
 67     AL  2020-03-08              1  30.050664
 68     AL  2020-03-09              0  30.195479
 69     AL  2020-03-10              0  30.495964
 70     AL  2020-03-11              0  31.195496
 71     AL  2020-03-12              0  31.500891
 ..    ...         ...            ...        ...
 138    AL  2020-05-18            218  45.939486
 139  

In [126]:
data_corr = pd.concat(lst,sort=True)
data_corr.head()

,date,indUF,new_confirmed,state
0,2020-03-17,40.871728,3,AC
1,2020-03-18,42.792925,0,AC
2,2020-03-19,45.103640,1,AC
3,2020-03-20,47.939821,3,AC
4,2020-03-21,50.940040,4,AC


In [127]:
data_corr.to_csv('data_corr.csv')

In [25]:
for s in mob.name.unique():
    
    print(s)
    
    #filtering the state in the datasets
    cases = series_state[series_state.state == s]
    
    mobs = mob[mob.name == s][(mob[mob.name == s].date2 >= cases.iloc[0,1])]
    
    # creating series of cummulative, log and etc
    
    cases['cumulative'] = cases.new_confirmed.cumsum()
    
    cases['cumLog'] = np.log(cases.filter(['cumulative']))
    
    cases['indUF'] = np.array(mobs.indUF)    
    
    cases['indUFLog'] = np.log(cases.filter(['indUF']))
    
    cases['new_confirmedLog'] = np.log(cases.filter(['new_confirmed']))
    
    cases = cases.replace([np.inf, -np.inf], np.nan).fillna(0)
    
    # test stationarity
    
    s1 = 'indUF'
    s2 = 'new_confirmed'
    
    # Differencing to get the data stationary
    
    casesDiff = cases.filter([s1,s2]).diff().dropna()
    
    casesDiff = casesDiff.apply(pd.to_numeric)
    
    print('\x1b[0;30;43m'+ 'stationarity of the series {}'.format(s1) + ' and {}'.format(s2) +'\x1b[0m')
    
    cases.filter([s1,s2]).apply(functools.partial(test_stationarity))
    
    print('\x1b[0;30;43m'+ 'stationarity of the differentiated series {}'.format(s1) + ' and {}'.format(s2)+'\x1b[0m')
    
    casesDiff.apply(functools.partial(test_stationarity))
    

AC
stationarity of the series indUF and new_confirmed
Results of Dickey-Fuller Test:
Test Statistic                 -2.306101
p-value                         0.170008
#Lags Used                      7.000000
Number of Observations Used    59.000000
Critical Value (1%)            -3.546395
Critical Value (5%)            -2.911939
Critical Value (10%)           -2.593652
dtype: float64
Results of Dickey-Fuller Test:
Test Statistic                  2.076605
p-value                         0.998765
#Lags Used                      8.000000
Number of Observations Used    58.000000
Critical Value (1%)            -3.548494
Critical Value (5%)            -2.912837
Critical Value (10%)           -2.594129
dtype: float64
stationarity of the differentiated series indUF and new_confirmed
Results of Dickey-Fuller Test:
Test Statistic                -6.449136e+00
p-value                        1.539055e-08
#Lags Used                     6.000000e+00
Number of Observations Used    5.900000e+01
Critica

<ipython-input-25-8da942ded2d4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumulative'] = cases.new_confirmed.cumsum()
<ipython-input-25-8da942ded2d4>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumLog'] = np.log(cases.filter(['cumulative']))
<ipython-input-25-8da942ded2d4>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

Test Statistic                 -3.270921
p-value                         0.016229
#Lags Used                      6.000000
Number of Observations Used    56.000000
Critical Value (1%)            -3.552928
Critical Value (5%)            -2.914731
Critical Value (10%)           -2.595137
dtype: float64
Results of Dickey-Fuller Test:
Test Statistic                 -4.964141
p-value                         0.000026
#Lags Used                      5.000000
Number of Observations Used    57.000000
Critical Value (1%)            -3.550670
Critical Value (5%)            -2.913766
Critical Value (10%)           -2.594624
dtype: float64
BA
stationarity of the series indUF and new_confirmed
Results of Dickey-Fuller Test:
Test Statistic                 -4.344613
p-value                         0.000371
#Lags Used                     10.000000
Number of Observations Used    67.000000
Critical Value (1%)            -3.531955
Critical Value (5%)            -2.905755
Critical Value (10%)           -2.

<ipython-input-25-8da942ded2d4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumulative'] = cases.new_confirmed.cumsum()
<ipython-input-25-8da942ded2d4>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumLog'] = np.log(cases.filter(['cumulative']))
<ipython-input-25-8da942ded2d4>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

stationarity of the series indUF and new_confirmed
Results of Dickey-Fuller Test:
Test Statistic                 -0.963641
p-value                         0.766293
#Lags Used                      7.000000
Number of Observations Used    64.000000
Critical Value (1%)            -3.536928
Critical Value (5%)            -2.907887
Critical Value (10%)           -2.591493
dtype: float64
Results of Dickey-Fuller Test:
Test Statistic                  0.893169
p-value                         0.993001
#Lags Used                      6.000000
Number of Observations Used    65.000000
Critical Value (1%)            -3.535217
Critical Value (5%)            -2.907154
Critical Value (10%)           -2.591103
dtype: float64
stationarity of the differentiated series indUF and new_confirmed
Results of Dickey-Fuller Test:
Test Statistic                 -5.070563
p-value                         0.000016
#Lags Used                      6.000000
Number of Observations Used    64.000000
Critical Value (1%)   

...

<ipython-input-25-8da942ded2d4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumulative'] = cases.new_confirmed.cumsum()
<ipython-input-25-8da942ded2d4>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumLog'] = np.log(cases.filter(['cumulative']))
<ipython-input-25-8da942ded2d4>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

Test Statistic                 -4.605642
p-value                         0.000126
#Lags Used                     11.000000
Number of Observations Used    67.000000
Critical Value (1%)            -3.531955
Critical Value (5%)            -2.905755
Critical Value (10%)           -2.590357
dtype: float64
Results of Dickey-Fuller Test:
Test Statistic                  4.457038
p-value                         1.000000
#Lags Used                     10.000000
Number of Observations Used    68.000000
Critical Value (1%)            -3.530399
Critical Value (5%)            -2.905087
Critical Value (10%)           -2.590001
dtype: float64
stationarity of the differentiated series indUF and new_confirmed
Results of Dickey-Fuller Test:
Test Statistic                 -2.201161
p-value                         0.205824
#Lags Used                      7.000000
Number of Observations Used    70.000000
Critical Value (1%)            -3.527426
Critical Value (5%)            -2.903811
Critical Value (10%)  

<ipython-input-25-8da942ded2d4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumulative'] = cases.new_confirmed.cumsum()
<ipython-input-25-8da942ded2d4>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumLog'] = np.log(cases.filter(['cumulative']))
<ipython-input-25-8da942ded2d4>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

Test Statistic                 -0.832206
p-value                         0.809478
#Lags Used                      7.000000
Number of Observations Used    64.000000
Critical Value (1%)            -3.536928
Critical Value (5%)            -2.907887
Critical Value (10%)           -2.591493
dtype: float64
Results of Dickey-Fuller Test:
Test Statistic                  0.599158
p-value                         0.987595
#Lags Used                      7.000000
Number of Observations Used    64.000000
Critical Value (1%)            -3.536928
Critical Value (5%)            -2.907887
Critical Value (10%)           -2.591493
dtype: float64
stationarity of the differentiated series indUF and new_confirmed
Results of Dickey-Fuller Test:
Test Statistic                -5.662133e+00
p-value                        9.327633e-07
#Lags Used                     6.000000e+00
Number of Observations Used    6.400000e+01
Critical Value (1%)           -3.536928e+00
Critical Value (5%)           -2.907887e+00
Crit

<ipython-input-25-8da942ded2d4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumulative'] = cases.new_confirmed.cumsum()
<ipython-input-25-8da942ded2d4>:16: RuntimeWarning: divide by zero encountered in log
  cases['cumLog'] = np.log(cases.filter(['cumulative']))
<ipython-input-25-8da942ded2d4>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumLog'] = np.log(cases.filter(['cumulative']))
<ipython-input-25-8da942ded2d4>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [26]:
for s in mob.name.unique():
    
    print(s)
    
    #filtering the state in the datasets
    cases = series_state[series_state.state == s]
    
    mobs = mob[mob.name == s][(mob[mob.name == s].date2 >= cases.iloc[0,1])]
    
    # creating series of cummulative, log and etc
    
    cases['cumulative'] = cases.new_confirmed.cumsum()
    
    cases['cumLog'] = np.log(cases.filter(['cumulative']))
    
    cases['indUF'] = np.array(mobs.indUF)    
    
    cases['indUFLog'] = np.log(cases.filter(['indUF']))
    
    cases['new_confirmedLog'] = np.log(cases.filter(['new_confirmed']))
    
    cases = cases.replace([np.inf, -np.inf], np.nan).fillna(0)
    
    
    # Analyse the lag dependence using Var
    
    s1 = 'indUF'
    s2 = 'new_confirmed'
    
    # Differencing to get the data stationary
    
    casesDiff = cases.filter([s1,s2]).diff().dropna()
    
    casesDiff = casesDiff.apply(pd.to_numeric)
    
    print('\x1b[0;30;43m'+ 'Var of the series {}'.format(s1) + ' and {}'.format(s2) +'\x1b[0m')

    model = VAR(cases.filter([s1,s2]))
    
    model1 = model.select_order()
    print(model1.summary())
    

AC
Var of the series indUF and new_confirmed
 VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0        9.789       9.862   1.784e+04       9.817
1        7.568       7.785       1937.       7.653
2        7.295       7.657       1474.       7.435
3        7.132      7.639*       1255.       7.328
4        7.259       7.910       1428.       7.511
5        7.355       8.151       1581.       7.664
6        7.415       8.355       1690.       7.780
7        7.413       8.498       1703.       7.834
8        7.251       8.480       1466.       7.728
9       6.591*       7.966      770.7*      7.124*
10       6.636       8.155       822.8       7.225
11       6.722       8.386       920.6       7.367
--------------------------------------------------
AL
Var of the series indUF and new_confirmed
 VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         

<ipython-input-26-c5b82fc0e335>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumulative'] = cases.new_confirmed.cumsum()
<ipython-input-26-c5b82fc0e335>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumLog'] = np.log(cases.filter(['cumulative']))
<ipython-input-26-c5b82fc0e335>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

 VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0        11.39       11.46   8.836e+04       11.42
1        8.332      8.553*       4156.       8.417
2        8.239       8.607       3790.      8.381*
3        8.328       8.844       4151.       8.527
4        8.188       8.851       3621.       8.444
5        8.259       9.069       3908.       8.572
6        8.129       9.087       3458.       8.498
7        8.149       9.254       3567.       8.576
8       7.989*       9.242      3082.*       8.472
9        8.052       9.451       3342.       8.591
10       8.139       9.686       3733.       8.735
11       8.121       9.815       3778.       8.775
--------------------------------------------------
GO
Var of the series indUF and new_confirmed
 VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         HQIC   
-------------------------------------

<ipython-input-26-c5b82fc0e335>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumulative'] = cases.new_confirmed.cumsum()
<ipython-input-26-c5b82fc0e335>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumLog'] = np.log(cases.filter(['cumulative']))
<ipython-input-26-c5b82fc0e335>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

Var of the series indUF and new_confirmed
 VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0        10.02       10.09   2.255e+04       10.05
1        5.908      6.116*       368.0       5.989
2        5.835       6.181       342.3      5.971*
3        5.815       6.300      336.1*       6.005
4        5.828       6.451       341.1       6.072
5        5.840       6.601       346.4       6.138
6        5.903       6.802       370.9       6.255
7        5.863       6.901       359.2       6.270
8       5.802*       6.978       341.0       6.263
9        5.869       7.184       369.4       6.384
10       5.897       7.350       386.0       6.467
11       5.930       7.521       406.6       6.553
--------------------------------------------------
RJ
Var of the series indUF and new_confirmed
 VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         HQI

<ipython-input-26-c5b82fc0e335>:22: RuntimeWarning: divide by zero encountered in log
  cases['new_confirmedLog'] = np.log(cases.filter(['new_confirmed']))
<ipython-input-26-c5b82fc0e335>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['new_confirmedLog'] = np.log(cases.filter(['new_confirmed']))
/Users/julianeoliveira/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
<ipython-input-26-c5b82fc0e335>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [16]:
for s in mob.name.unique():
    
    print(s)
    
    #filtering the state in the datasets
    cases = series_state[series_state.state == s]
    
    mobs = mob[mob.name == s][(mob[mob.name == s].date2 >= cases.iloc[0,1])]
    
    # creating series of cummulative, log and etc
    
    cases['cumulative'] = cases.new_confirmed.cumsum()
    
    cases['cumLog'] = np.log(cases.filter(['cumulative']))
    
    cases['indUF'] = np.array(mobs.indUF)    
    
    cases['indUFLog'] = np.log(cases.filter(['indUF']))
    
    cases['new_confirmedLog'] = np.log(cases.filter(['new_confirmed']))
    
    cases = cases.replace([np.inf, -np.inf], np.nan).fillna(0)
    
    
    # calculate correlation between series without lag and any previous analysis
    
    s1 = 'indUF'
    s2 = 'new_confirmedLog'
    
    print('\x1b[0;30;43m'+ 'Crosscorrelation between the series {}'.format(s1) + ' and {}'.format(s2) +'\x1b[0m')

    
    r, p = stats.pearsonr(cases.dropna()[s1], cases.dropna()[s2])
    print(f"Scipy computed Pearson r: {r} and p-value: {p}")
    
    r, p = stats.spearmanr(cases.dropna()[s1], cases.dropna()[s2])
    print(f"Scipy computed Spearman r: {r} and p-value: {p}")
    
    r, p = stats.kendalltau(cases.dropna()[s1], cases.dropna()[s2])
    print(f"Scipy computed Kendall's tau: {r} and p-value: {p}")

AC
Crosscorrelation between the series indUF and new_confirmedLog
Scipy computed Pearson r: -0.23273933288939955 and p-value: 0.05805012189316953
Scipy computed Spearman r: -0.34937564445894287 and p-value: 0.0037571225003498673
Scipy computed Kendall's tau: -0.2615526243690664 and p-value: 0.002117024021865508
AL
Crosscorrelation between the series indUF and new_confirmedLog
Scipy computed Pearson r: 0.2041683722721967 and p-value: 0.07687955839437262
Scipy computed Spearman r: 0.014853729460754413 and p-value: 0.8986612441834195
Scipy computed Kendall's tau: 0.022247872417182707 and p-value: 0.7843109791221065
AM
Crosscorrelation between the series indUF and new_confirmedLog
Scipy computed Pearson r: 0.41645925548453977 and p-value: 0.00030331869957842244
Scipy computed Spearman r: -0.07257753850998798 and p-value: 0.5475148125910658
Scipy computed Kendall's tau: -0.09418039738457037 and p-value: 0.2472603132579939
AP
Crosscorrelation between the series indUF and new_confirmedLog
Sci

<ipython-input-16-f4604f3030ed>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumulative'] = cases.new_confirmed.cumsum()
<ipython-input-16-f4604f3030ed>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumLog'] = np.log(cases.filter(['cumulative']))
<ipython-input-16-f4604f3030ed>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda


Scipy computed Pearson r: -0.8273823710375054 and p-value: 1.975247993807203e-17
Scipy computed Spearman r: -0.8233160013091596 and p-value: 3.852319633781322e-17
Scipy computed Kendall's tau: -0.6477127238021891 and p-value: 2.7349427308047455e-14
GO
Crosscorrelation between the series indUF and new_confirmedLog
Scipy computed Pearson r: -0.1447303492272528 and p-value: 0.22513626351625338
Scipy computed Spearman r: -0.23543285615113463 and p-value: 0.04649946755817708
Scipy computed Kendall's tau: -0.23560950418472518 and p-value: 0.003926035325795329
MA
Crosscorrelation between the series indUF and new_confirmedLog
Scipy computed Pearson r: -0.08567132880589005 and p-value: 0.5008826273049979
Scipy computed Spearman r: -0.0280990233900752 and p-value: 0.8255555992364424
Scipy computed Kendall's tau: -0.023904572186687872 and p-value: 0.7808590525752981
MG
Crosscorrelation between the series indUF and new_confirmedLog
Scipy computed Pearson r: 0.318279269258661 and p-value: 0.005079

<ipython-input-16-f4604f3030ed>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumulative'] = cases.new_confirmed.cumsum()
<ipython-input-16-f4604f3030ed>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumLog'] = np.log(cases.filter(['cumulative']))
<ipython-input-16-f4604f3030ed>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

Scipy computed Spearman r: -0.10467491343682254 and p-value: 0.38154200116646075
Scipy computed Kendall's tau: -0.09269513763326002 and p-value: 0.25114207691059565
RJ
Crosscorrelation between the series indUF and new_confirmedLog
Scipy computed Pearson r: 0.6677779477299195 and p-value: 1.7921839018292786e-11
Scipy computed Spearman r: 0.027726848039585868 and p-value: 0.8083461586589481
Scipy computed Kendall's tau: -0.1103342076254781 and p-value: 0.15410530050353594
RN
Crosscorrelation between the series indUF and new_confirmedLog
Scipy computed Pearson r: 0.13386303067566102 and p-value: 0.2622706519008658
Scipy computed Spearman r: -0.08000212890579624 and p-value: 0.5041132616536474
Scipy computed Kendall's tau: -0.07126784938460416 and p-value: 0.3895073048826255
RO
Crosscorrelation between the series indUF and new_confirmedLog
Scipy computed Pearson r: -0.7417418144169322 and p-value: 2.3579134703969624e-12
Scipy computed Spearman r: -0.8099479432459623 and p-value: 5.30571501

<ipython-input-16-f4604f3030ed>:20: RuntimeWarning: divide by zero encountered in log
  cases['new_confirmedLog'] = np.log(cases.filter(['new_confirmed']))
<ipython-input-16-f4604f3030ed>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['new_confirmedLog'] = np.log(cases.filter(['new_confirmed']))
<ipython-input-16-f4604f3030ed>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumulative'] = cases.new_confirmed.cumsum()
<ipython-input-16-f4604f3030ed>:14: SettingWithCopyWarning: 
A value is t

In [17]:
for s in mob.name.unique():
    
    print(s)
    
    #filtering the state in the datasets
    cases = series_state[series_state.state == s]
    
    mobs = mob[mob.name == s][(mob[mob.name == s].date2 >= cases.iloc[0,1])]
    
    # creating series of cummulative, log and etc
    
    cases['cumulative'] = cases.new_confirmed.cumsum()
    
    cases['cumLog'] = np.log(cases.filter(['cumulative']))
    
    cases['indUF'] = np.array(mobs.indUF)    
    
    cases['indUFLog'] = np.log(cases.filter(['indUF']))
    
    cases['new_confirmedLog'] = np.log(cases.filter(['new_confirmed']))
    
    cases = cases.replace([np.inf, -np.inf], np.nan).fillna(0)
    
    
    # croscorrelation with diferent lags 
    
    s1 = 'indUF'
    s2 = 'new_confirmedLog'
    
    print('\x1b[0;30;43m'+ 'Crosscorrelation with lags between the series {}'.format(s1) + ' and {}'.format(s2) +'\x1b[0m')
    
    xcov = [crosscorr(cases[s1], cases[s2], lag=i) for i in range(40)]
    print(xcov)
    
    
    #x= cases.indUF
    #y= cases.new_confirmedLog

    #n =len(x)
    #l1 = (-1/n) + (2/math.sqrt(n))

    #fig, [ax1,ax2] = plt.subplots(3, 1)
    #ax1.xcorr(x, y, usevlines=True, maxlags=None, normed=True, lw=2)

    #ax1.axhline(l1, color='blue', linestyle='dashed', lw=1)

    #ax1.grid(True)

    #ax2.acorr(x, usevlines=True, normed=True, maxlags=None, lw=2)
    #ax2.grid(True)
    
    #ax2.scatter(cases.new_confirmedLog,cases.indUF, marker='o')
    #ax2.legend('new_confirmedLog','indUF')
    
    #ax3.plot(xcov, marker='o')
    #ax3.axhline(l1, color='blue', linestyle='dashed', lw=1)
    #ax3.axhline(-l1, color='blue', linestyle='dashed', lw=1)
    #ax3.axis(ymin=-1,ymax=1)

    #plt.show()

AC
Crosscorrelation with lags between the series indUF and new_confirmedLog
[-0.23273933288939957, -0.3188580632553643, -0.3939077041249942, -0.41611593752039255, -0.43411592990535564, -0.44030843190817237, -0.42692149117053885, -0.37728834764123115, -0.31956893339072984, -0.26440633277591524, -0.20920765713655262, -0.1529388520911468, -0.05305132112177263, 0.012373720203069833, 0.08121241136583526, 0.10610560027933369, 0.10469191592349425, 0.10400155902334096, 0.14966162742500116, 0.20700584505594258, 0.2628322002304386, 0.2303952683597338, 0.2889888481460554, 0.32925747768761765, 0.3772449193721644, 0.43057658701373025, 0.4900931176866494, 0.46025935121866307, 0.5365049749142, 0.556873204604772, 0.6023277012075817, 0.5978804285310967, 0.5890827988885741, 0.5267585371669512, 0.5234328493846239, 0.5318344789362275, 0.48232622959306365, 0.3685751381007373, 0.3958521668673371, 0.27697152078398324]
AL
Crosscorrelation with lags between the series indUF and new_confirmedLog
[0.204168372272

<ipython-input-17-ef53d769bdfd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumulative'] = cases.new_confirmed.cumsum()
<ipython-input-17-ef53d769bdfd>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['cumLog'] = np.log(cases.filter(['cumulative']))
<ipython-input-17-ef53d769bdfd>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[0.059481674277924466, -0.07068813034861365, -0.2184920118756019, -0.3872217218174582, -0.5124313152774812, -0.6017061324120785, -0.6393907739481399, -0.653570997311886, -0.6530188743893853, -0.6471823903765875, -0.6029943090214973, -0.5731250932675848, -0.5343729437860506, -0.4716854607386468, -0.3893922881534799, -0.30099149067791403, -0.20367117156135492, -0.14460614303058894, -0.07298883329445252, -0.038532710223175476, -0.00807102752978713, 0.01036853905126585, 0.0006263099131926143, 0.02147378531329265, 0.020895706386931525, 0.0359477480634123, 0.12118259764324389, 0.17191331376408106, 0.2484025060425132, 0.32884075469841806, 0.315285339598054, 0.3662121318997267, 0.32824090132095673, 0.32046225948698753, 0.31868786597904436, 0.3498763997356067, 0.375085061930486, 0.4549999053938047, 0.49021824147234844, 0.5728867811926428]
DF
Crosscorrelation with lags between the series indUF and new_confirmedLog
[0.3352109790456924, 0.3146790192718561, 0.2831121362340283, 0.2420302073938229, 0

<ipython-input-17-ef53d769bdfd>:20: RuntimeWarning: invalid value encountered in log
  cases['new_confirmedLog'] = np.log(cases.filter(['new_confirmed']))



[0.30199363500490095, 0.24207650419672636, 0.16796923186900933, 0.08260749781058604, -0.016631858329270874, -0.135236515484966, -0.27450479349379725, -0.4333185255109878, -0.5841867863914626, -0.6622167490010182, -0.6873362711657475, -0.6770662551581565, -0.6612513395347688, -0.6364638865437051, -0.6166449715288059, -0.5834115682977503, -0.5613051135841749, -0.513914901501777, -0.4936937814020768, -0.45508564393343137, -0.43318491985365604, -0.37764372538350477, -0.35491346870798196, -0.3555434724089592, -0.32858790991860337, -0.30533855574158314, -0.26885814711457945, -0.21740754556320516, -0.19860970021510438, -0.16518269347166953, -0.1047919798316707, -0.05912555695053811, 0.039427547000659584, 0.08915944862197914, 0.1276381677644037, 0.1619373990745069, 0.22435348020699725, 0.28748358823116743, 0.3604959505658994, 0.36351220964391157]
PI
Crosscorrelation with lags between the series indUF and new_confirmedLog
[-0.6645502659080548, -0.725543559624907, -0.7501343797145137, -0.737484

Crosscorrelation with lags between the series indUF and new_confirmedLog
[0.49345834220101403, 0.4650072251339868, 0.43700558977523846, 0.4139509254888714, 0.3889867782303593, 0.3606017581968518, 0.3272310644069175, 0.2902263203736604, 0.25066572783923435, 0.19784953171836353, 0.13428302791940072, 0.07171713190283546, -0.003977979908481235, -0.08671471161880648, -0.19065065015878688, -0.30774575356519684, -0.41413078233075984, -0.5347938525057944, -0.6512644831729788, -0.7387732464865777, -0.8033976747689968, -0.8276068701345151, -0.8170726178091334, -0.8133051124254226, -0.7919513668149888, -0.8051589226553193, -0.80526247203427, -0.7965783594080654, -0.7917680231552571, -0.7944933005232577, -0.8022224236422334, -0.814991297456207, -0.7900589265554588, -0.7615925481123094, -0.7393672120055275, -0.7269591056709718, -0.7531254651586945, -0.7356556358492845, -0.7361436317487046, -0.7071901548085707]
TO
Crosscorrelation with lags between the series indUF and new_confirmedLog
[-0.490244514

<ipython-input-17-ef53d769bdfd>:14: RuntimeWarning: divide by zero encountered in log
  cases['cumLog'] = np.log(cases.filter(['cumulative']))


# Analysis of the fit results

## Data results

In [ ]:
import sys
import glob

path = r'/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/results/seiir_bootstrap'
filenames = glob.glob(path + "/*.csv")
filenames

In [ ]:
lst_df = []
for f in filenames:
    print(f)
    df = pd.read_csv(f, low_memory = False, encoding="iso-8859-1")     
    #df.columns= df.columns.str.lower()
    lst_df.append(df)

In [ ]:
df = pd.concat(lst_df,sort=True)
df.head()

## Calculation of the confidence intervals

In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.median(a), scipy.stats.sem(a)
    h = se * scipy.stats.skewnorm.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

def day_date(t):
    t_date = []
    for days in t:
        days = np.nan_to_num(days)
        date = datetime.datetime(2020, 1, 1) + datetime.timedelta(int(days) - 1)
        t_date.append(date)

    return(t_date)

def r0(b,delta,gammaA=1/3.5,gammaS=1/4,p=0.2):

    R0a = b*delta/gammaA

    R0s = b/gammaS

    R0_geral = p*R0s + (1-p)*R0a
    
    return(R0_geral)

In [ ]:
dfs = []
for uf in df.state.unique():
    for area in df.type.unique():
    
        frame = df[(df.state == uf) & (df.type == area)]


        beta0= np.array(mean_confidence_interval(frame['beta0'], confidence=0.95))

        beta0Round = np.round(mean_confidence_interval(frame['beta0'], confidence=0.95), decimals=2)

        beta1 = np.round(mean_confidence_interval(frame['beta1'], confidence=0.95), decimals=2)

        beta2 = np.round(mean_confidence_interval(frame['beta2'], confidence=0.95), decimals=2)

        delta = np.array(mean_confidence_interval(frame['delta'], confidence=0.95))

        deltaRound = np.round(mean_confidence_interval(frame['delta'], confidence=0.95), decimals=2)

        tcut0 = np.round(mean_confidence_interval(frame['tcut0'], confidence=0.95), decimals=0)

        tcut1 = np.round(mean_confidence_interval(frame['tcut1'], confidence=0.95), decimals=0)

        tcut0_date = np.array(day_date(tcut0))

        tcut1_date = np.array(day_date(tcut1))

        R0 = np.round(r0(beta0,delta),2)
    
        data = np.array([beta0, beta1,delta,tcut0,tcut0_date,tcut1,tcut1_date,R0])
        df1 = pd.DataFrame(data=data, index=['beta0','beta1','delta','tcut0','tcut0_date','tcut1','tcut1_date','R0'],columns=['median','lower','upper']).transpose()
        df1['state'] = uf
        df1['type'] = area
        dfs.append(df1)

In [ ]:
dfs

In [ ]:
dfs_result = pd.concat(dfs,sort=True)

In [ ]:
#dfs_result.to_csv('/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/results/ic_par.csv')

# Comparing dates 

In [37]:
pars = pd.read_csv('/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/results/ic_par.csv')

In [38]:
pars.head()

,Unnamed: 0,R0,beta0,beta1,delta,state,tcut0,tcut0_date,tcut1,tcut1_date,type
0,median,1.91,1.141240,1.08,0.312494,TO,128.0,2020-05-07,NaN,2019-12-31,estado
1,lower,1.71,1.071549,1.02,0.282560,TO,125.0,2020-05-04,NaN,2019-12-31,estado
2,upper,2.13,1.210931,1.13,0.342427,TO,131.0,2020-05-10,NaN,2019-12-31,estado
3,median,1.35,0.964620,0.88,0.213145,TO,129.0,2020-05-08,NaN,2019-12-31,capital
4,lower,1.16,0.882606,0.79,0.183743,TO,125.0,2020-05-04,NaN,2019-12-31,capital


In [51]:
pars.tcut0_date =  pd.to_datetime(pars.tcut0_date)

In [41]:
series_state.head()

,state,date,new_confirmed
0,AC,2020-03-17,3
1,AC,2020-03-18,0
2,AC,2020-03-19,1
3,AC,2020-03-20,3
4,AC,2020-03-21,4


In [53]:
series_state.date =  pd.to_datetime(series_state.date)

In [82]:
for s in series_state.state.unique():

    #filtering the state in the datasets
    cases = series_state[series_state.state == s]
    
    p = pars[pars.state == s]
    
    mobs = mob[mob.name == s]
    mobs = mobs[(mobs[mobs.name == s].date2 >= cases.iloc[0,1])]
    
    mobs[mobs.indUF == max(mobs.indUF)]    
    
    d1 = cases.iloc[0,1]
    
    d2 = p.iloc[0,7]
    
    
    print(s, d1, d2, 'tcut1 and first case ->', abs((d2 - d1).days))
    
    #mobs = mob[mob.name == s][(mob[mob.name == s].date2 >= cases.iloc[0,1])]

AC 2020-03-17 00:00:00 2020-05-17 00:00:00 tcut1 and first case -> 61
AL 2020-03-08 00:00:00 2020-04-28 00:00:00 tcut1 and first case -> 51
AM 2020-03-13 00:00:00 2020-04-06 00:00:00 tcut1 and first case -> 24
AP 2020-03-20 00:00:00 2020-04-09 00:00:00 tcut1 and first case -> 20
BA 2020-03-06 00:00:00 2020-03-30 00:00:00 tcut1 and first case -> 24
CE 2020-03-16 00:00:00 2020-04-02 00:00:00 tcut1 and first case -> 17
DF 2020-03-07 00:00:00 2020-03-23 00:00:00 tcut1 and first case -> 16
ES 2020-03-19 00:00:00 2020-04-14 00:00:00 tcut1 and first case -> 26
GO 2020-03-12 00:00:00 2020-04-04 00:00:00 tcut1 and first case -> 23
MA 2020-03-20 00:00:00 2020-04-15 00:00:00 tcut1 and first case -> 26
MG 2020-03-08 00:00:00 2020-03-22 00:00:00 tcut1 and first case -> 14
MS 2020-03-14 00:00:00 2020-05-06 00:00:00 tcut1 and first case -> 53
MT 2020-03-20 00:00:00 2020-04-04 00:00:00 tcut1 and first case -> 15
PA 2020-03-18 00:00:00 2020-04-20 00:00:00 tcut1 and first case -> 33
PB 2020-03-18 00:00:

In [78]:
mob

,state,name,indUF,indCAP,indInland,date,date2
0,12,AC,33.924855,34.104342,33.770914,01-Mar-2020 00:00:00,2020-03-01
1,12,AC,34.429550,34.609595,34.275130,02-Mar-2020 00:00:00,2020-03-02
2,12,AC,34.919644,35.118317,34.749249,03-Mar-2020 00:00:00,2020-03-03
3,12,AC,35.147736,35.333082,34.988770,04-Mar-2020 00:00:00,2020-03-04
4,12,AC,34.982379,35.166184,34.824735,05-Mar-2020 00:00:00,2020-03-05
...,...,...,...,...,...,...,...
2392,17,TO,40.665150,41.608438,40.454475,18-May-2020 00:00:00,2020-05-18
2393,17,TO,40.623109,41.506788,40.425748,19-May-2020 00:00:00,2020-05-19
2394,17,TO,40.477544,41.327409,40.287735,20-May-2020 00:00:00,2020-05-20
2395,17,TO,40.207128,41.051061,40.018643,21-May-2020 00:00:00,2020-05-21


In [66]:
s = 'SC'
mobs = mob[mob.name == s][(mob[mob.name == s].date2 >= cases.iloc[0,1])]
    
mobs[mobs.indUF == max(mobs.indUF)]

,state,name,indUF,indCAP,indInland,date,date2
2070,42,SC,63.448656,61.557285,63.619165,24-Mar-2020 00:00:00,2020-03-24


In [67]:
series_state[series_state.state == 'SC']

,state,date,new_confirmed
1605,SC,2020-03-12,2
1606,SC,2020-03-13,1
1607,SC,2020-03-14,2
1608,SC,2020-03-15,1
1609,SC,2020-03-16,1
...,...,...,...
1672,SC,2020-05-18,399
1673,SC,2020-05-19,238
1674,SC,2020-05-20,86
1675,SC,2020-05-21,111


In [65]:
pars[pars.state == 'SC']

,Unnamed: 0,R0,beta0,beta1,delta,state,tcut0,tcut0_date,tcut1,tcut1_date,type
180,median,3.99,1.798053,0.64,0.507495,SC,84.0,2020-03-24,NaN,2019-12-31,estado
181,lower,3.79,1.761289,0.61,0.481869,SC,83.0,2020-03-23,NaN,2019-12-31,estado
182,upper,4.21,1.834817,0.66,0.533120,SC,84.0,2020-03-24,NaN,2019-12-31,estado
183,median,1.43,0.789553,1.86,0.361720,SC,138.0,2020-05-17,NaN,2019-12-31,capital
184,lower,1.25,0.726145,1.76,0.330207,SC,134.0,2020-05-13,NaN,2019-12-31,capital
185,upper,1.62,0.852962,1.95,0.393234,SC,141.0,2020-05-20,NaN,2019-12-31,capital
186,median,3.78,1.785287,0.68,0.471016,SC,84.0,2020-03-24,NaN,2019-12-31,interior
187,lower,3.57,1.742275,0.65,0.445733,SC,83.0,2020-03-23,NaN,2019-12-31,interior
188,upper,4.00,1.828299,0.71,0.496299,SC,85.0,2020-03-25,NaN,2019-12-31,interior


# ...


In [ ]:
sig_uf = []
for value in mob.state_name:
    if value == 'Minas Gerais': value = 'MG'
    elif value == 'Ceará': value = 'CE'
    elif value == 'Pernambuco': value = 'PE'
    elif value == 'Mato Grosso do Sul': value = 'MS'
    elif value == 'Rio Grande do Sul': value = 'RS'
    elif value == 'Sergipe': value = 'SE'
    elif value == 'Rio Grande do Norte': value = 'RN'
    elif value == 'Acre': value = 'AC'
    elif value == 'Alagoas': value = 'AL'
    elif value == 'Bahia': value = 'BA'
    elif value == 'Rondônia': value = 'RO'
    elif value == 'Rio de Janeiro': value = 'RJ'
    elif value == 'Piauí': value = 'PI'
    elif value == 'Paraná': value = 'PR'
    elif value == 'Espírito Santo': value = 'ES'
    elif value == 'Pará': value = 'PA'
    elif value == 'Amapá': value = 'AP'
    elif value == 'Mato Grosso': value = 'MT'
    elif value == 'Paraíba': value = 'PB'
    elif value == 'Roraima': value = 'RR'
    elif value == 'Goiás': value = 'GO'
    elif value == 'São Paulo': value = 'SP'
    elif value == 'Distrito Federal': value = 'DF'
    elif value == 'Maranhão': value = 'MA'
    elif value == 'Santa Catarina': value = 'SC'
    elif value == 'Tocantins': value = 'TO'
    elif value == 'Amazonas': value = 'AM'
    sig_uf.extend([value])